In [ ]:
# Comments:
'''
- Add the thickness calculation;
- Add the ask_stop control.
'''

# Libraries:
from os import listdir
from os.path import join, exists
from chardet import detect
from numpy import ones, array, linspace, exp, log10, tan, pi
from scipy.signal import find_peaks
from pandas import read_csv, DataFrame, ExcelWriter
from matplotlib.pyplot import subplots, show

# Classes:

# Function:
## Introduction to the program:
def intro():
    # Present the program:
    print('--------------------------------')
    print('Welcome to "Analise de Capilar"!')
    print('--------------------------------')
    print()
    print('Here you will be able to analyse capillaries.')
    print()
## Find the path to the sample files: 
def get_sample_folder():
    # Read data:
    while True:
        with open("Gerenciador de Caminho.txt", mode='rt') as f:
            folder = f.read()
            f.close()
        try: 
            if not exists(folder):
                raise FileExistsError('There is no folder with such path!')
            else:
                break
        except FileExistsError as fee:
            print(fee)
            print('Go to the "Gerenciador de Caminho.txt" and change its content!')
            input('Type anything when you are done: ')
    # Get the sample's folder path:
    global sample_folder
    while True:
        # Get the sample's number:
        while True:
            n_sample = input('Sample\'s number: ')
            try:
                if not n_sample.lstrip('-').isdigit():
                    raise ValueError('It is not a number!')
                elif '-' in n_sample:
                    raise ValueError('Negative integers are not allowed!')
                elif ',' in n_sample:
                    raise ValueError('Not integer numbers are not allowed!')
                n_sample = int(n_sample)
                if n_sample == 0:
                    raise ValueError('There is no fiber label 0!')
                else:
                    break
            except ValueError as ve:
                print(ve)
        # Join the folder with the sample identification suffix:
        sample_folder = join(folder, f'E - {n_sample}')
        try:
            if not exists(sample_folder):
                raise FileExistsError("Invalid fiber type or sample's number!")
            else:
                break
        except FileExistsError as fee:
            print(fee)
            print('Try again.')
## Data file modification:
def modify_file(data_path):
    file_name = data_path.split('E - ')[1].split('\\')[1]
    top = '00\t\n'
    bottom = '"CTRWL",1050.00\n'
    header = 'Wavelength [nm],Transmission [dB]\n'
    # Get the file enconding:
    with open(data_path, mode='rb') as f:
        detection = detect(f.read())
        f.close()
    # Read the data:
    with open(data_path, mode='rt', encoding=detection['encoding']) as f:
        rows = f.readlines()
        f.close()
    ## The file need to be modified:
    if top in rows:
        print('It need to be modifed!')
        # Find the header position:
        ind = [i for i, row in enumerate(rows) if row == top]
        ind = ind[0]
        # Cut off the data above the header and substitute by the new header:
        data = str()
        for i, row in enumerate(rows):
            if row == bottom:
                break
            elif i > ind:
                data += row.replace(' ', '')
        data = header + data
        # Modify the old file:
        with open(data_path, mode='wt') as g:
            g.write(data)
            g.close()
        print(f'Modification on "{file_name}" is done.')
    ## The file was already modified:
    else:
        print(f'"{file_name}" was already modified.')
## Individual analysis:
def individual_analysis(data_path, t_list):
    # General parameters:
    band = data_path.split('\\')[-1].split('Banda ')[1].split('.TXT')[0]
    D_ext = 200 # Fiber external diameter [um].
    n2 = 1.000 # Air.
    L = 8.5 # Fiber length [cm].
    L /= 1e4 # cm -> um.
    j01 = 2.405 # First zero of 1st type Bessel function.
    # Mathematical functions:
    ## Refractive index:
    f_n = lambda l: (1 + 1.1819 * l**2 / (l**2 - 0.011313))**0.5 # The wavelength shall be in um.
    lx = linspace(0.450, 1.750, 100)
    ly = f_n(lx)
    ### Plot:
    fig, ax = subplots(nrows=1, ncols=1, layout='constrained', figsize=(12,4))
    ax.set_xlabel('Wavelength [nm]', loc='center', fontsize=12)
    ax.set_ylabel('Refractive index', loc='center', fontsize=12)
    ax.scatter(lx * 1e3, ly, c='black', s=30)
    show()
    # Wavelength of minimum transmission:
    f_wv = lambda l, m, t: 2 * t / m * (f_n(l)**2 - n2**2)**0.5 # Thickness and wavelength shall be in um.
    # Data manipulation:
    df = read_csv(data_path)
    ## Remove the unwanted data:
    df = df[df['Wavelength [nm]'] >= 450] 
    df = df[df['Transmission [dB]'] >= -100] 
    # Plot to visualize the data:
    fig, ax = subplots(nrows=1, ncols=1, layout='constrained', figsize=(12,4))
    ax.set_xlabel('Wavelength [nm]', loc='center', fontsize=12)
    ax.set_ylabel('Transmission [dB]', loc='center', fontsize=12)
    ax.scatter(df['Wavelength [nm]'], df['Transmission [dB]'], c='black', s=30)
    show()
    # Find a valley:
    ## Select an interval in the data frame:
    df_m = df[df['Wavelength [nm]'] >= 1400] 
    df_m = df_m[df_m['Wavelength [nm]'] <= 1450] 
    peaks, _ = find_peaks(-df_m['Transmission [dB]'], width=8)
    lamb_m = df_m.iat[peaks[0], 0] # Wavelength of minimum transmission for calibration.
    lamb_m /= 1e3 # nm -> um.
    ## Plot:
    fig, ax = subplots(nrows=1, ncols=1, layout='constrained', figsize=(12,4))
    ax.set_xlabel('Wavelength [nm]', loc='center', fontsize=12)
    ax.set_ylabel('Transmission [dB]', loc='center', fontsize=12)
    ax.scatter(df_m['Wavelength [nm]'], df_m['Transmission [dB]'], c='black', s=30)
    ax.plot(lamb_m * 1e3 * ones(2), array([min(df_m['Transmission [dB]']), max(df_m['Transmission [dB]'])]), c='red', lw=2, linestyle='dashed')
    show()
    for t in t_list:
        print(f't = {t}um')
        # Calculate the m-th order of the previous valley:
        print(f'Wavelength to calculate the m-th order: {lamb_m * 1e3}nm')
        print(f'Corresponding refractive index: {f_n(lamb_m)}')
        m_cal = round(2 * t / lamb_m * (f_n(lamb_m)**2 - n2**2)**0.5) # m for calibration.
        print(f'Chosen m: {m_cal}')
        ## Calculate the nearby m orders:
        l1, l2 = 5, 5
        m_list = [value for value in range(m_cal - l1, m_cal + l2 + 1)]
        # Find the theoretical peaks:
        lamb_list = list()
        N_i = 20 # Number of iterations of the numerical equation solver.
        for m in m_list:
            # Solve numerically:
            l = lamb_m # Initial guess.
            for i in range(N_i):
                l = f_wv(l, m, t)
            lamb_list.append(l)
        df_lamb = DataFrame(data={'Minimum wavelength [nm]': lamb_list})
        with ExcelWriter(join(sample_folder, f'Tabela Minimos Banda {band} - t = {t}um.xlsx'), engine='openpyxl') as writer:
            df_lamb.to_excel(writer, index=False)
        # Plot in wavelength:
        ## The entire spectrum:
        fig, ax = subplots(nrows=1, ncols=1, layout='constrained', figsize=(12,4))
        ax.set_xlabel('Wavelength [nm]', loc='center', fontsize=12)
        ax.set_ylabel('Transmission [dB]', loc='center', fontsize=12)
        ax.scatter(df['Wavelength [nm]'], df['Transmission [dB]'], c='black', s=30)
        ind_m = 0
        for i, (m, lamb) in enumerate(zip(m_list, lamb_list)):
            if m > m_cal:
                color = 'blue'
            elif m < m_cal:
                color = 'red'
            else:
                color = 'green'
                ind_m = i
            ax.plot(lamb * 1e3 * ones(2), array([min(df['Transmission [dB]']), max(df['Transmission [dB]'])]), c=color, linewidth=2, linestyle='dashed')
        show()
        ## Select a region:
        fig, ax = subplots(nrows=1, ncols=1, layout='constrained', figsize=(12,4))
        ax.set_xlabel('Wavelength [nm]', loc='center', fontsize=12)
        ax.set_ylabel('Transmission [dB]', loc='center', fontsize=12)
        ax.set_xlim(left=1300, right=1700)
        ax.set_ylim(bottom=-80, top=-55)
        ax.scatter(df['Wavelength [nm]'], df['Transmission [dB]'], c='black', s=30)
        for i, lamb in enumerate(lamb_list):
            if i > ind_m:
                color = 'blue' 
            elif i < ind_m:
                color = 'red'
            else:
                color = 'green'
            ax.plot(lamb * 1e3 * ones(2), array([min(df['Transmission [dB]']), max(df['Transmission [dB]'])]), c=color, linewidth=2, linestyle='dashed')
        show()
        # Change wavelength to frequency:
        c = 3e8 # Speed of the light in vaccumm (m/s). 
        c *= 1e6 # m/s -> um/s.
        df['Frequency [Hz]'] = c / (df['Wavelength [nm]'] / 1e3) # Wavelength is in um.
        freq_list = [c / lamb for lamb in lamb_list]
        # Plot in wavelength:
        ## The entire spectrum:
        fig, ax = subplots(nrows=1, ncols=1, layout='constrained', figsize=(12,4))
        ax.set_xlabel('Frequency [THz]', loc='center', fontsize=12)
        ax.set_ylabel('Transmission [dB]', loc='center', fontsize=12)
        ax.scatter(df['Frequency [Hz]'] / 1e12, df['Transmission [dB]'], c='black', s=30)
        for i, freq in enumerate(freq_list):
            if i > ind_m:
                color = 'blue' 
            elif i < ind_m:
                color = 'red'
            else:
                color = 'green'
            ax.plot(freq * ones(2) / 1e12, array([min(df['Transmission [dB]']), max(df['Transmission [dB]'])]), c=color, linewidth=2, linestyle='dashed')
        show()
        ## Select a region:
        fig, ax = subplots(nrows=1, ncols=1, layout='constrained', figsize=(12,4))
        ax.set_xlabel('Frequency [THz]', loc='center', fontsize=12)
        ax.set_ylabel('Transmission [dB]', loc='center', fontsize=12)
        ax.set_xlim(left=180, right=230)
        ax.set_ylim(bottom=-85, top=-55)
        ax.scatter(df['Frequency [Hz]'] / 1e12, df['Transmission [dB]'], c='black', s=30)
        for i, freq in enumerate(freq_list):
            if i > ind_m:
                color = 'blue' 
            elif i < ind_m:
                color = 'red'
            else:
                color = 'green'
            ax.plot(freq * ones(2) / 1e12, array([min(df['Transmission [dB]']), max(df['Transmission [dB]'])]), c=color, linewidth=2, linestyle='dashed')
        show()   
        # Test:
        R = D_ext / 2 - t # Core radius [um].
        P_dB_points = list()
        fig, ax = subplots()
        l_space = linspace(1300, 1700, 100000)
        for l in l_space:
            l /= 1e3 # nm -> um.
            eps = (f_n(l) / n2)**2
            k0 = 2 * pi / l
            phi = k0 * t * (f_n(l)**2 - n2**2)**0.5
            alpha = ((1 + (1 / tan(phi))**2) / (eps - 1)) * (j01**3 / (k0**3 * n2**3 * R**4)) * ((eps**2 + 1) / 2) # [um^-1].
            P_dB = 10 * log10(exp(-alpha * L))
            P_dB_points.append(P_dB)
        P_dB_points = array(P_dB_points)
        ax.plot(l_space, P_dB_points)
        ax.set_xlim(left=1300, right=1700)
        show()
## Main control function:
def main():
    # Introduction:
    intro()
    # Find sample folder:
    get_sample_folder()
    # Access files withing sample folder:
    path_list = [file for file in listdir(sample_folder) if 'Dados Espectro' in file]
    print('Files to be analysed:')
    [print('"'+file+'"') for file in path_list]
    # For each spectrum:
    for file in path_list:
        data_path = join(sample_folder, file)
        # Modify data files:
        modify_file(data_path)
        # List of capillary thickness:
        t_list = [22]
        # t_list = [22.136, 22.787, 24.740, 25.391, 26.043] # um.
        # Individual analysis:
        individual_analysis(data_path, t_list)

# Run the program:
main()

In [5]:
# Given a list of orders m, calculate t to have an agreement between experimental and theoretical lambda_m:
n = lambda l: (1 + 1.1819 * l**2 / (l**2 - 0.011313))**0.5
l = 1.4196
m_try = [18,19,20]
for m in m_try:
    t = m * l / (2 * (n(l)**2 - 1)**0.5)
    print(f'm = {m}:')
    print(f't = {t}um')
    print()

m = 18:
t = 11.71913910975964um

m = 19:
t = 12.370202393635177um

m = 20:
t = 13.021265677510712um



# Test code